# Molecular Docking example

Most of this code comes from [here](https://chem-workflows.com/articles/2021/09/18/1-molecular-docking/?fbclid=IwAR0CgvxmSlGgBsUuTX7nHh4r4SryAA9I9PeYovPrHk7te0jTWDFwxVq-snA)

In [1]:
SDF_FILE = '../molecules/human_serum_albumin_CHEMBL_data.sdf'
PDB_FILE = '../molecules/4la0_human_serum_albumin_cleaned_without_ligand.pdb'

## The necessary imports

In [2]:
import sys
sys.path.append('..')

from sbap import pipeline

In [3]:
from pymol import cmd
import py3Dmol

from vina import Vina

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

from meeko import MoleculePreparation
from meeko import obutils

import sys, os
sys.path.insert(1, '../external')
from utils import fix_protein, getbox, generate_ledock_file, pdbqt_to_sdf, dok_to_sdf

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import prolif as plf
from prolif.plotting.network import LigNetwork

import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False

We will place all our temporary files in the 'working' directory not tracked by git

In [4]:
if not os.path.isdir('../working'):
    os.mkdir('../working')
os.chdir('../working')

### Ligand preparation

In [5]:
mol= [m for m in pybel.readfile(filename=SDF_FILE,format='sdf')][148]  # just a random one
mol.addh()
out=pybel.Outputfile(filename='ligandH.mol2',format='mol2',overwrite=True)
out.write(mol)
out.close()

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warnin

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warnin

First problem - why are there those errors? How big a difference does it make?

In [6]:
! python3 ../external/mk_prepare_ligand.py -i ligandH.mol2 -o ligandH.pdbqt

An alternative - I didn't get it to work

In [7]:
#sdf_file = Chem.SDMolSupplier(SDF_FILE)
#sample_ligand = sdf_file[248]
#ligand = Chem.AddHs(sample_ligand)
#AllChem.EmbedMolecule(ligand, AllChem.ETKDG())
#writer = Chem.rdmolfiles.SDWriter('../molecules/ligandH.sdf')
#writer.write(ligand)
! #python3 ../external/mk_prepare_ligand.py -i ../molecules/ligandH.sdf -o ../molecules/ligand.pdbqt

### Protein (receptor) preparation

In [8]:
fix_protein(filename=PDB_FILE, addHs_pH=7.4, try_renumberResidues=True, output='proteinH.pdb')

There is a mysterious parameter:
- addHs_pH: float ; Add hydrogens at user defined pH

I copied the value which was in their code, have no idea how to choose the correct pH...

In [9]:
! obabel -ipdb proteinH.pdb -opdbqt -h

REMARK  Name = proteinH.pdb
REMARK  2044 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: _1  and  _5
REMARK    2  A    between atoms: _5  and  _7
REMARK    3  A    between atoms: _5  and  _9
REMARK    4  A    between atoms: _13  and  _15
REMARK    5  A    between atoms: _15  and  _17
REMARK    6  A    between atoms: _15  and  _19
REMARK    7  A    between atoms: _19  and  _22
REMARK    8  A    between atoms: _30  and  _32
REMARK    9  A    between atoms: _32  and  _34
REMARK   10  A    between atoms: _32  and  _36
REMARK   11  A    between atoms: _36  and  _39
REMARK   12  A    between atoms: _39  and  _42
REMARK   13  A    between atoms: _42  and  _45
REMARK   14  A    between atoms: _45  and  _48
REMARK   15  A    between atoms: _52  and  _54
REMARK   16  A    between atoms: _54  and  _56
REMARK   17  A    between atoms: _54  and  _58
REMARK   18  A    between atoms: _58  and  _61
REMARK   19  A    between atoms: _63  and  _65
REMA

REMARK  1208  A    between atoms: _5276  and  _5279
REMARK  1209  A    between atoms: _5290  and  _5294
REMARK  1210  A    between atoms: _5294  and  _5297
REMARK  1211  A    between atoms: _5321  and  _5325
REMARK  1212  A    between atoms: _5325  and  _5328
REMARK  1213  A    between atoms: _5328  and  _5330
REMARK  1214  A    between atoms: _5333  and  _5337
REMARK  1215  A    between atoms: _5337  and  _5340
REMARK  1216  A    between atoms: _5349  and  _5350
REMARK  1217  A    between atoms: _5354  and  _5358
REMARK  1218  A    between atoms: _5358  and  _5361
REMARK  1219  A    between atoms: _5365  and  _5369
REMARK  1220  A    between atoms: _5369  and  _5371
REMARK  1221  A    between atoms: _5369  and  _5375
REMARK  1222  A    between atoms: _5381  and  _5385
REMARK  1223  A    between atoms: _5385  and  _5387
REMARK  1224  A    between atoms: _5385  and  _5391
REMARK  1225  A    between atoms: _5397  and  _5401
REMARK  1226  A    between atoms: _5401  and  _5404
REMARK  1227

ROOT
ATOM      1  CA  ILE A 289      12.524 -11.100   4.652  0.00  0.00    +0.162 C 
ENDROOT
BRANCH   1   2
ATOM      2  C   ILE A 289      12.042  -9.752   4.063  0.00  0.00    +0.234 C 
ATOM      3  O   ILE A 289      12.826  -8.811   3.886  0.00  0.00    -0.274 OA
ATOM      4  N   ALA A 290      10.745  -9.653   3.780  0.00  0.00    -0.303 N 
ATOM      5  H   ALA A 290       9.915 -10.476   3.914  0.00  0.00    +0.149 HD
BRANCH   4   6
ATOM      6  CA  ALA A 290      10.184  -8.421   3.224  0.00  0.00    +0.156 C 
ATOM      7  CB  ALA A 290       8.784  -8.659   2.687  0.00  0.00    +0.038 C 
BRANCH   6   8
ATOM      8  C   ALA A 290      10.185  -7.250   4.217  0.00  0.00    +0.234 C 
ATOM      9  N   GLU A 291      10.081  -7.514   5.521  0.00  0.00    -0.302 N 
ATOM     10  H   GLU A 291      10.043  -8.520   6.138  0.00  0.00    +0.150 HD
ATOM     11  O   ALA A 290      10.290  -6.101   3.795  0.00  0.00    -0.274 OA
BRANCH   9  12
ATOM     12  CA  GLU A 291      10.005  -6.414 

ATOM    460  C   GLU A 375      21.229  19.794  -5.647  0.00  0.00    +0.234 C 
ATOM    461  N   PHE A 376      21.738  18.825  -4.875  0.00  0.00    -0.302 N 
ATOM    462  H   PHE A 376      22.884  18.570  -4.925  0.00  0.00    +0.150 HD
ATOM    463  O   GLU A 375      20.016  20.026  -5.718  0.00  0.00    -0.274 OA
BRANCH 461 464
ATOM    464  CA  PHE A 376      20.880  17.878  -4.146  0.00  0.00    +0.163 C 
BRANCH 464 465
ATOM    465  C   PHE A 376      20.444  18.511  -2.841  0.00  0.00    +0.234 C 
ATOM    466  O   PHE A 376      19.416  18.138  -2.277  0.00  0.00    -0.274 OA
ATOM    467  N   LYS A 377      21.213  19.496  -2.382  0.00  0.00    -0.302 N 
ATOM    468  H   LYS A 377      22.383  19.536  -2.567  0.00  0.00    +0.150 HD
BRANCH 467 469
ATOM    469  CA  LYS A 377      20.979  20.108  -1.085  0.00  0.00    +0.159 C 
BRANCH 469 470
ATOM    470  C   LYS A 377      19.548  20.602  -0.892  0.00  0.00    +0.237 C 
ATOM    471  O   LYS A 377      18.906  20.199   0.075  0.00

BRANCH10851086
ATOM   1086  C   ASP A 493     -11.212  27.470  -4.873  0.00  0.00    +0.234 C 
ATOM   1087  O   ASP A 493     -12.136  27.376  -4.058  0.00  0.00    -0.274 OA
ATOM   1088  N   GLU A 494     -10.993  28.560  -5.600  0.00  0.00    -0.302 N 
ATOM   1089  H   GLU A 494      -9.999  28.679  -6.244  0.00  0.00    +0.150 HD
BRANCH10881090
ATOM   1090  CA  GLU A 494     -11.833  29.733  -5.483  0.00  0.00    +0.159 C 
BRANCH10901091
ATOM   1091  C   GLU A 494     -13.009  29.600  -6.404  0.00  0.00    +0.234 C 
ATOM   1092  O   GLU A 494     -14.061  30.188  -6.168  0.00  0.00    -0.274 OA
ATOM   1093  N   THR A 495     -12.839  28.769  -7.426  0.00  0.00    -0.300 N 
ATOM   1094  H   THR A 495     -11.769  28.339  -7.660  0.00  0.00    +0.150 HD
BRANCH10931095
ATOM   1095  CA  THR A 495     -13.819  28.646  -8.489  0.00  0.00    +0.187 C 
BRANCH10951096
ATOM   1096  C   THR A 495     -15.025  27.876  -7.991  0.00  0.00    +0.236 C 
ATOM   1097  N   TYR A 496     -14.854  27.20

...

ATOM   5017  CE1 TYR A 149       3.343 -10.189  -3.871  0.00  0.00    +0.045 A 
ATOM   5018  CD1 TYR A 149       2.146  -9.599  -3.465  0.00  0.00    +0.007 A 
BRANCH50165019
ATOM   5019  OH  TYR A 149       5.551 -10.924  -3.311  0.00  0.00    -0.507 OA
ATOM   5020  HH  TYR A 149       6.431 -10.179  -3.034  0.00  0.00    +0.292 HD
ENDBRANCH50165019
ENDBRANCH50125013
ENDBRANCH35955012
BRANCH35685021
ATOM   5021  CB  PHE A 155       1.669 -14.407   5.341  0.00  0.00    +0.067 C 
BRANCH50215022
ATOM   5022  CG  PHE A 155       2.354 -14.321   6.654  0.00  0.00    -0.045 A 
ATOM   5023  CD2 PHE A 155       2.590 -15.475   7.388  0.00  0.00    +0.004 A 
ATOM   5024  CE2 PHE A 155       3.193 -15.421   8.634  0.00  0.00    +0.000 A 
ATOM   5025  CZ  PHE A 155       3.541 -14.212   9.178  0.00  0.00    +0.000 A 
ATOM   5026  CE1 PHE A 155       3.285 -13.048   8.487  0.00  0.00    +0.000 A 
ATOM   5027  CD1 PHE A 155       2.671 -13.099   7.219  0.00  0.00    +0.004 A 
ENDBRANCH50215022
END

ATOM   5165  CD  ARG A 143     -16.005 -10.372  -1.155  0.00  0.00    +0.196 C 
BRANCH51655166
ATOM   5166  NE  ARG A 143     -17.238 -10.025  -1.846  0.00  0.00    -0.279 N 
ATOM   5167  CZ  ARG A 143     -18.401  -9.831  -1.240  0.00  0.00    +0.341 C 
ATOM   5168  NH2 ARG A 143     -18.498  -9.953   0.076  0.00  0.00    -0.292 N 
ATOM   5169 HH21 ARG A 143     -18.492  -8.923   0.667  0.00  0.00    +0.255 HD
ATOM   5170 HH22 ARG A 143     -19.369 -10.560   0.613  0.00  0.00    +0.255 HD
ATOM   5171  NH1 ARG A 143     -19.460  -9.510  -1.959  0.00  0.00    -0.292 N 
ATOM   5172 HH11 ARG A 143     -20.407  -9.139  -1.335  0.00  0.00    +0.255 HD
ATOM   5173 HH12 ARG A 143     -20.038  -9.993  -2.878  0.00  0.00    +0.255 HD
ATOM   5174  HE  ARG A 143     -17.287 -10.279  -2.997  0.00  0.00    +0.258 HD
ENDBRANCH51655166
ENDBRANCH51645165
ENDBRANCH51635164
ENDBRANCH36185163
BRANCH36825175
ATOM   5175  CB  PHE A 126     -13.297 -20.726  14.531  0.00  0.00    +0.067 C 
BRANCH51755176
ATO

ATOM   5336  CB  VAL A 240       7.778 -11.584  -7.448  0.00  0.00    +0.012 C 
ATOM   5337  CG2 VAL A 240       6.688 -11.830  -6.398  0.00  0.00    +0.009 C 
ATOM   5338  CG1 VAL A 240       8.860 -12.680  -7.393  0.00  0.00    +0.009 C 
ENDBRANCH31295336
BRANCH31685339
ATOM   5339  CB  VAL A 230      21.419  -4.865  -5.382  0.00  0.00    +0.012 C 
ATOM   5340  CG2 VAL A 230      22.081  -4.818  -3.972  0.00  0.00    +0.009 C 
ATOM   5341  CG1 VAL A 230      20.240  -3.903  -5.522  0.00  0.00    +0.009 C 
ENDBRANCH31685339
BRANCH32305342
ATOM   5342  CB  VAL A 215      17.746  -0.120  -7.750  0.00  0.00    +0.012 C 
ATOM   5343  CG2 VAL A 215      19.184   0.284  -7.509  0.00  0.00    +0.009 C 
ATOM   5344  CG1 VAL A 215      17.660  -1.100  -8.935  0.00  0.00    +0.009 C 
ENDBRANCH32305342
BRANCH31525345
ATOM   5345  CB  VAL A 234      16.293  -5.227  -8.721  0.00  0.00    +0.012 C 
ATOM   5346  CG2 VAL A 234      16.617  -4.381  -7.549  0.00  0.00    +0.009 C 
ATOM   5347  CG1 VAL 

ATOM   5498  OE2 GLU A 229      22.323 -12.570  -4.524  0.00  0.00    -0.481 OA
ATOM   5499  HE2 GLU A 229      21.506 -12.222  -4.224  0.00  0.00    +0.295 HD
ENDBRANCH54965498
ENDBRANCH54955496
ENDBRANCH54945495
ENDBRANCH31735494
BRANCH31915500
ATOM   5500  CB  LYS A 224      26.807  -5.917   3.314  0.00  0.00    +0.032 C 
BRANCH55005501
ATOM   5501  CG  LYS A 224      26.957  -5.220   4.667  0.00  0.00    +0.000 C 
BRANCH55015502
ATOM   5502  CD  LYS A 224      27.175  -6.226   5.790  0.00  0.00    -0.019 C 
BRANCH55025503
ATOM   5503  CE  LYS A 224      27.238  -5.541   7.132  0.00  0.00    -0.086 C 
BRANCH55035504
ATOM   5504  NZ  LYS A 224      28.270  -4.484   7.127  0.00  0.00    +0.378 N 
ATOM   5505  HZ1 LYS A 224      27.984  -3.589   7.871  0.00  0.00    -0.090 HD
ATOM   5506  HZ2 LYS A 224      29.113  -4.987   7.819  0.00  0.00    -0.090 HD
ATOM   5507  HZ3 LYS A 224      28.971  -4.049   6.266  0.00  0.00    -0.090 HD
ENDBRANCH55035504
ENDBRANCH55025503
ENDBRANCH55015502

BRANCH56645665
ATOM   5665  CG  GLU A 265      19.299 -21.704  -0.189  0.00  0.00    +0.123 C 
BRANCH56655666
ATOM   5666  CD  GLU A 265      18.761 -22.328  -1.466  0.00  0.00    +0.305 C 
ATOM   5667  OE1 GLU A 265      19.024 -21.795  -2.582  0.00  0.00    -0.251 OA
BRANCH56665668
ATOM   5668  OE2 GLU A 265      18.057 -23.353  -1.343  0.00  0.00    -0.481 OA
ATOM   5669  HE2 GLU A 265      17.892 -23.709  -0.492  0.00  0.00    +0.295 HD
ENDBRANCH56665668
ENDBRANCH56655666
ENDBRANCH56645665
ENDBRANCH29335664
BRANCH29255670
ATOM   5670  CB  GLN A 267      22.067 -15.727   4.293  0.00  0.00    +0.041 C 
BRANCH56705671
ATOM   5671  CG  GLN A 267      21.225 -15.033   5.261  0.00  0.00    +0.098 C 
BRANCH56715672
ATOM   5672  CD  GLN A 267      20.777 -15.951   6.353  0.00  0.00    +0.211 C 
ATOM   5673  OE1 GLN A 267      20.750 -15.559   7.534  0.00  0.00    -0.276 OA
ATOM   5674  NE2 GLN A 267      20.376 -17.174   5.975  0.00  0.00    -0.329 N 
ATOM   5675 HE21 GLN A 267      21.266

The [ADFR Suite](https://ccsb.scripps.edu/adfr/) helper programs looked promising, but I had this error 'cannot find reduce_wwPDB_het_dict.txt', no idea what this file does, but it is HUGE, like 1.3milion lines

In [10]:
! #/home/justyna/ADFRsuite-1.0/bin/reduce {PDB_FILE} > proteinH.pdb
! #/home/justyna/ADFRsuite-1.0/bin/prepare_receptor -r proteinH.pdb -o protein.pdbqt

## Docking
- Step 1. calculate box
- Step 2. perform docking using Vina

In [11]:
cmd.load(filename='proteinH.pdb',format='pdb',object='prot')
cmd.load(filename='ligandH.mol2',format='mol2',object='lig')

center, size= getbox(selection='lig',extending=5.0,software='vina')

cmd.delete('all')

print(center,'\n',size)

{'center_x': 5.259149968624115, 'center_y': 1.2372499704360962, 'center_z': 0.0} 
 {'size_x': 19.122099995613098, 'size_y': 20.0106999874115, 'size_z': 10.0}


In [12]:
v = Vina(sf_name='vina')

v.set_receptor('protein.pdbqt')

v.set_ligand_from_file('ligandH.pdbqt')

v.compute_vina_maps(center=[center['center_x'], center['center_y'], center['center_z']], 
                    box_size=[size['size_x'], size['size_y'], size['size_z']])

# Dock the ligand
v.dock(exhaustiveness=10, n_poses=10)
v.write_poses('docking_output.pdbqt', n_poses=10, overwrite=True)

Computing Vina grid ... done.
Performing docking (random seed: 487585763) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


## Visualisation

In [13]:
pdbqt_to_sdf(pdbqt_file='docking_output.pdbqt', output='docking_output.sdf')
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('proteinH.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open('ligandH.mol2','r').read(),format='sdf')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

results=Chem.SDMolSupplier('docking_output.sdf')

p=Chem.MolToMolBlock(results[0],False)

print('Reference: Magenta | Vina Pose: Cyan')
print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

Reference: Magenta | Vina Pose: Cyan
Pose: 1 | Score: -11.902


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Attempt at calculating interaction fingerprint

In [14]:
# load protein
prot = mda.Universe("proteinH.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

# load ligands
lig_suppl = list(plf.sdf_supplier('docking_output.sdf'))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
results_df = fp.to_dataframe(return_atoms=True)
results_df

  0%|          | 0/7 [00:00<?, ?it/s]

ligand               UNL1                                             
protein          ARG222.A      GLN196.A      LEU238.A      LYS195.A   
interaction    VdWContact    VdWContact    VdWContact    VdWContact   
Frame                                                                 
0            (None, None)  (None, None)  (None, None)        (7, 0)  \
1            (None, None)  (None, None)  (None, None)  (None, None)   
2                  (8, 0)  (None, None)  (None, None)        (1, 0)   
3            (None, None)  (None, None)  (None, None)        (3, 0)   
4            (None, None)  (None, None)        (3, 0)  (None, None)   
5            (None, None)  (None, None)        (6, 0)  (None, None)   
6            (None, None)       (33, 0)        (0, 0)  (None, None)   

ligand                                   
protein          LYS199.A      PHE211.A  
interaction    VdWContact    VdWContact  
Frame                                    
0                  (0, 0)  (None, None)  
1                 (14, 0)       (33, 0)  
2                  (6, 0)       (32, 0)  
3                  (3, 0)  (None, None)  
4            (None, None)  (None, None)  
5            (None, None)  (None, None)  
6            (None, None)  (None, None)